## 1. Import Module & Data Preparation

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import log_loss, precision_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from itertools import combinations

train = pd.read_csv('final_model_train_input.csv')
test = pd.read_csv('final_model_test_input.csv')
X_train = train.drop(['is_duplicate'],  axis=1)
y_train = train['is_duplicate']

X_test = test.drop(['is_duplicate'],  axis=1)
y_test = test['is_duplicate']

model_list = X_train.columns.tolist()
input_list = []
for i in range(2, len(model_list)):
    input_list += [",".join(map(str, comb)) for comb in combinations(model_list, i)]

# 2. Stepwise Tuning

In [2]:
#result_df = pd.Dataframe(columns = ['Input', 'Train Log Loss', 'Trai Precision', 'Test Log Loss', 'Test Precision'])
result_list = []
log_clf = LogisticRegression()
for i in input_list:
    subset = i.split(",")
    train_subset = X_train[subset]
    test_subset = X_test[subset]
    log_clf.fit(train_subset,y_train)
    preds_train = log_clf.predict(train_subset)
    preds_prob_train = log_clf.predict_proba(train_subset)
    preds_test = log_clf.predict(test_subset)
    preds_prob_test = log_clf.predict_proba(test_subset)
    l = [i, log_loss(y_train, preds_prob_train),precision_score(y_train, preds_train), 
                       log_loss(y_test, preds_prob_test), precision_score(y_test, preds_test)]
    result_list.append(l)

result_df = pd.DataFrame(result_list)
result_df.columns = ['Stacked Model','Train Log Loss', 'Train Precision', 'Test Log Loss', 'Test Precision']
result_df = result_df.sort_values(by =['Test Log Loss'], ascending = True)
result_df.reset_index(drop = True, inplace = True)
result_df

,Stacked Model,Train Log Loss,Train Precision,Test Log Loss,Test Precision
0,"bert,lgbm",0.125318,0.935907,0.133022,0.930700
1,"bert,lgbm,mlp",0.125301,0.935968,0.133088,0.930564
2,"bert,mlp",0.133120,0.930157,0.135653,0.927189
3,"lgbm,mlp",0.131216,0.936443,0.144879,0.927349
4,"bert,xgboost",0.099880,0.946260,0.172028,0.934690
5,"lgbm,xgboost",0.120456,0.937038,0.180276,0.921289
6,"bert,lgbm,xgboost",0.099365,0.946301,0.180886,0.934214
7,"bert,rf",0.075601,0.959547,0.219718,0.894265
8,"bert,rf,xgboost",0.074951,0.960062,0.225169,0.886706
9,"bert,lgbm,rf",0.070369,0.963516,0.250165,0.880541


# 3. Best Subset Selection

In [3]:
best_subset_feature = result_df['Stacked Model'].values[0].split(",")
X_train = X_train[best_subset_feature]
X_test = X_test[best_subset_feature]

In [4]:
log_clf.fit(X_train, y_train)
preds_train = log_clf.predict(X_train)
preds_prob_train = log_clf.predict_proba(X_train)
preds_test = log_clf.predict(X_test)
preds_prob_test = log_clf.predict_proba(X_test)

In [5]:
print("The train log loss is:", log_loss(y_train, preds_prob_train))
print("The train precision is:", precision_score(y_train, preds_train))
print("The test log loss is:", log_loss(y_test, preds_prob_test))
print("The test precision is:", precision_score(y_test, preds_test))

The train log loss is: 0.12531842132839877
The train precision is: 0.9359066709721222
The test log loss is: 0.13302161760440695
The test precision is: 0.9307002505604642
